In [1]:
!wget https://hktn2022.blob.core.windows.net/dataset/hist_data.csv
!wget https://hktn2022.blob.core.windows.net/dataset/test.csv

--2022-03-27 07:42:08--  https://hktn2022.blob.core.windows.net/dataset/hist_data.csv
Resolving hktn2022.blob.core.windows.net (hktn2022.blob.core.windows.net)... 52.239.143.36
Connecting to hktn2022.blob.core.windows.net (hktn2022.blob.core.windows.net)|52.239.143.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347659751 (332M) [application/vnd.ms-excel]
Saving to: ‘hist_data.csv’

hist_data.csv       100%[===================>] 331.55M  9.09MB/s    in 33s     

2022-03-27 07:42:41 (10.1 MB/s) - ‘hist_data.csv’ saved [347659751/347659751]

--2022-03-27 07:42:42--  https://hktn2022.blob.core.windows.net/dataset/test.csv
Resolving hktn2022.blob.core.windows.net (hktn2022.blob.core.windows.net)... 52.239.143.36
Connecting to hktn2022.blob.core.windows.net (hktn2022.blob.core.windows.net)|52.239.143.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76848171 (73M) [application/vnd.ms-excel]
Saving to: ‘test.csv’

test.csv            10

In [2]:
import pandas as pd
import numpy as np
from collections import Counter 
import itertools

In [3]:
## Загружаем данные
hist_df = pd.read_csv('hist_data.csv')

In [5]:
## Для каждого продукта считаем количество корзин в которые он вошел
item_cart = hist_df.groupby('item_id', as_index=False)['count'].count()

In [6]:
## Аналогично для каждого пользователя вычисляем сколько раз он совершал покупки
user_cart = hist_df.groupby('buyer_id', as_index=False)['pav_order_id'].nunique()

In [7]:
n = 5

In [8]:
## Для каждого товара находим товары с которыми он чаще всего попадает в корзину
tmp_df = hist_df[['item_id', 'pav_order_id']].sort_values(['item_id', 'pav_order_id'])
tmp_df = tmp_df.merge(tmp_df, how='left', on=['pav_order_id'], suffixes=('', '_left'))
tmp_df = tmp_df.loc[tmp_df['item_id'] != tmp_df['item_id_left']]

In [9]:
item_commons_df = tmp_df.groupby(['item_id'], as_index=False)['item_id_left'].agg(lambda x: Counter(x).most_common(n))

In [10]:
## Для каждого клиента находим товары которые он покупает чаще всего
buyer_commons_df = hist_df.groupby(['buyer_id'], as_index=False)['item_id'].agg(lambda x: Counter(x).most_common(2 * n))

In [11]:
test_df = pd.read_csv('test.csv')

In [12]:
pred_df = test_df.groupby(['pav_order_id', 'buyer_id'])['item_id'].agg([('basket', list)]).reset_index()

In [15]:
## Для совстречаемых товаров находим вероятность увидеть их в одной корзине
item_commons_df = item_commons_df.merge(item_cart, on=['item_id'])
item_commons_df['item_id_left'] = item_commons_df.apply(lambda x : [(y[0], y[1] / x['count']) for y in x['item_id_left']], 1)

In [16]:
def preds_for_basket(row):
    basket = row['basket']
    user_id = row['buyer_id']
    common_items = item_commons_df[item_commons_df['item_id'].isin(basket)].copy()

    ## Взешиваем вероятности товаров в зависимости от того как часто они были купленны
    common_items['weight'] = np.exp(-(common_items['count'].max() - common_items['count']) / (common_items['count'].mean()))
    common_items['item_id_left'] = common_items.apply(lambda x : [(y[0], y[1] * x['weight']) for y in x['item_id_left']], 1)
    common_items = common_items['item_id_left'].values.tolist()
    common_items = list(itertools.chain(*common_items))
    common_items = list(zip(*common_items))
    res_df = pd.DataFrame({'item_id' : common_items[0], 'freq' : common_items[1], 'user' : 0})

    ## Добавляем товары наиболее популярные у пользователя
    if user_id in buyer_commons_df['buyer_id'].unique():
        user_items = buyer_commons_df.loc[buyer_commons_df['buyer_id'] == user_id, 'item_id'].values[0]
        user_items = list(zip(*user_items))

        res_tmp_df = pd.DataFrame({'item_id' : user_items[0], 'freq' : user_items[1], 'user' : 1})
        res_tmp_df['freq'] = res_tmp_df['freq'] / user_cart.loc[user_cart['buyer_id'] == user_id, 'pav_order_id'].values[0]
        res_df = pd.concat([res_df, res_tmp_df])

    ## Удаляем из рекомендации товары уже добавленные в корзины
    res_df = res_df[~res_df['item_id'].isin(basket)]
    ## Находим суммарные вероятности для товаров и возвращаем 20 самых популярных
    return res_df.groupby('item_id', as_index=False).agg({'freq' : sum, 'user' : max}).sort_values(['freq'], ascending=False).head(20)['item_id'].values.tolist()

In [17]:
from tqdm import tqdm
tqdm.pandas()

pred_df['preds'] = pred_df[['basket', 'buyer_id']].progress_apply(preds_for_basket, axis=1)

100%|██████████| 80244/80244 [18:06<00:00, 73.86it/s]


In [18]:
pred_df[['pav_order_id', 'preds']].to_csv('preds.csv', index=False)